In [1]:
from netwin import *
import os
import matplotlib.pyplot as plt

NameError: name 'self' is not defined

In [2]:
root_dir = '/home/chaggar/Documents/Network_Inference'
graph_path = os.path.join(root_dir, 'data/brain_networks/scale1.csv')


In [3]:
class NetworkDiffusion(Model):

    def f(self, p, t, theta):
        k = theta
        du = k * (-self.L() @ p)
        return du

    def solve(self, p, t, theta):
        return odeint(self.f, p, t, args=(theta,))

    def forward(self, u0, t): 
        p = np.exp(u0[:-1])
        theta = u0[-1]
        #
        # print(u0)
        u = self.solve(p, t, theta) 
        return u.T 

In [4]:
m = NetworkDiffusion(network_path=graph_path)

In [5]:
p = np.ones([83])
p[40] = 10
k = 5

u0 = np.append(np.log(p), k)

t = np.linspace(0,0.1,100)

p0 = np.ones([83])
k0 = 1

u_0 = np.append(p0, k0)

In [6]:
sim = m.f(p = p, t = t, theta = k)

In [7]:
data = m.forward(u0=u0, t=t)

In [8]:
problem = InferenceProblem(inference='VB', model=m, data = data, t=t, init_means=u_0)

In [16]:
sol = vb(problem, n=10)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Finished!


In [10]:
fkpp = NetworkFKPP(network_path=graph_path)

In [11]:
p = np.ones([83])
p[40] = 40
k = 5
a = 1

u0 = np.append(np.log(p), [k, a])

t = np.linspace(0,1,100)

p0 = np.ones([83])
k0 = 2
a0 = 3
u_0 = np.append(p0, [k0, a0])

In [12]:
sim = fkpp.f(p = p, t = t, theta = [k, a])

In [13]:
data = fkpp.forward(u0=u0, t=t)

In [14]:
problem = InferenceProblem(inference='VB', model=fkpp, data = data, t=t, init_means=u_0)

In [15]:
sol = vb(problem, n=10)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Finished!
